In [3]:
import cv2
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import numpy as np
import os

# === CONFIG: Update this to your actual image directory ===
IMAGE_DIR = "img"  # Put your images inside a folder named 'ysma_images'

# Load available image filenames
def load_image_files():
    supported_exts = (".jpg", ".jpeg", ".png", ".bmp")
    return [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(supported_exts)]

# Edge detection function
def apply_edge_detection(image, method):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if method == "Sobel X":
        return cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    elif method == "Sobel Y":
        return cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    elif method == "Laplacian":
        return cv2.Laplacian(gray, cv2.CV_64F)
    elif method == "Canny":
        return cv2.Canny(gray, 100, 200)
    else:
        return gray

# GUI function to process selected image
def process_image():
    selected_file = image_var.get()
    method = method_var.get()

    if not selected_file or not method:
        messagebox.showerror("Error", "Please select both an image and an edge detection method.")
        return

    image_path = os.path.join(IMAGE_DIR, selected_file)
    image = cv2.imread(image_path)

    if image is None:
        messagebox.showerror("Error", f"Failed to load image: {selected_file}")
        return

    result = apply_edge_detection(image, method)
    if method != "Canny":
        result = cv2.convertScaleAbs(result)

    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_display = ImageTk.PhotoImage(Image.fromarray(img_rgb).resize((300, 300)))
    result_display = ImageTk.PhotoImage(Image.fromarray(result).resize((300, 300)))

    original_label.config(image=img_display)
    original_label.image = img_display

    edge_label.config(image=result_display)
    edge_label.image = result_display

# === GUI SETUP ===
root = tk.Tk()
root.title("YSMA Edge Detection Viewer")
root.geometry("750x550")

# Load image filenames
image_files = load_image_files()

# Image selection
tk.Label(root, text="Select an Object (Image):").pack(pady=5)
image_var = tk.StringVar()
ttk.Combobox(root, textvariable=image_var, values=image_files, width=40).pack()

# Edge detection method
tk.Label(root, text="Select Edge Detection Method:").pack(pady=5)
method_var = tk.StringVar()
ttk.Combobox(root, textvariable=method_var, values=["Sobel X", "Sobel Y", "Laplacian", "Canny"]).pack()

# Apply button
tk.Button(root, text="Apply Edge Detection", command=process_image).pack(pady=10)

# Image display area
frame = tk.Frame(root)
frame.pack()

original_label = tk.Label(frame)
original_label.grid(row=0, column=0, padx=10, pady=10)

edge_label = tk.Label(frame)
edge_label.grid(row=0, column=1, padx=10, pady=10)

root.mainloop()
